# 提取照片经纬度Extract FoodPlace photo latitude and longitude


In [30]:
import pandas as pd

import os

import exifread
from fractions import Fraction

## 解析经纬度Parsing longitude and latitude

In [31]:
def get_coordinates(file_path):
    '''
    输入图片文件路径，输出经纬度
    '''
    try:
        with open(file_path, 'rb') as f:
            tags = exifread.process_file(f)
            
            # 打印所有 EXIF 标签以进行调试
            print("EXIF 标签:", tags)
            
            # 检查是否存在 GPS 标签
            if 'GPS GPSLatitude' not in tags or 'GPS GPSLongitude' not in tags:
                print("缺少 GPS 信息")
                return None, None, None
            
            Latitude_list = tags['GPS GPSLatitude'].values
            Longitude_list = tags['GPS GPSLongitude'].values
            LatitudeRef = tags['GPS GPSLatitudeRef'].values
            LongitudeRef = tags['GPS GPSLongitudeRef'].values
            Time = tags.get('EXIF DateTimeOriginal', '未知时间').values
            
            # 检查纬度和经度列表的长度
            print("纬度列表:", Latitude_list)
            print("经度列表:", Longitude_list)
            
            if len(Latitude_list) < 3 or len(Longitude_list) < 3:
                print("纬度或经度数据不完整")
                return None, None, None
            
            # 处理纬度
            Latitude = convert_to_degrees(Latitude_list)
            # 根据纬度参考值确定纬度符号
            if LatitudeRef != 'N':
                Latitude = -Latitude
            
            # 处理经度
            Longitude = convert_to_degrees(Longitude_list)
            # 根据经度参考值确定经度符号
            if LongitudeRef != 'E':
                Longitude = -Longitude
            
            return Latitude, Longitude, Time
    except FileNotFoundError:
        print(f"文件未找到: {file_path}")
        return None, None, None
    except KeyError as e:
        print(f"缺少 EXIF 标签: {e}")
        return None, None, None
    except Exception as e:
        print(f"发生错误: {e}")
        return None, None, None

def convert_to_degrees(value):
    """将分数形式的 GPS 数据转换为度数"""
    # 处理度
    d = float(Fraction(value[0]))

    # 处理分
    if len(value) > 1:
        m = float(Fraction(value[1]))
    else:
        m = 0  # 如果没有分，设为 0

    # 处理秒
    if len(value) > 2:
        s = float(Fraction(value[2]))
    else:
        s = 0  # 如果没有秒，设为 0

    return d + (m / 60.0) + (s / 3600.0)

# # 调用函数
# result = get_coordinates('2024-11-02_154929.jpg')
# if result:
#     Latitude, Longitude, Time = result
#     print('纬度:', Latitude, '经度:', Longitude, '时间:', Time)
# else:
#     print("未能获取坐标信息")

## 遍历每张照片，获取所有照片经纬度信息Traverse each photo and get the latitude and longitude information of all photos

In [23]:
print(os.listdir('.'))

['.ipynb_checkpoints', '1.getLatitudeLongtitudeFromPhotos-Copy1.ipynb', '2.DrawMap-Copy1.ipynb', '2.DrawMap.ipynb', 'FileName&Infomation.csv', 'FoodPlace.csv', 'HTMLMAP.html', 'images', 'Step1getTransportaionLatitudeLongtitudeFromPhotos-Copy1.ipynb', 'Step2getFoodPlaceLatitudeLongtitudeFromPhotos.ipynb', 'Transportation.csv']


In [24]:
# 进入images目录
os.chdir('images')
df_list = []  # 创建一个列表来存储每个文件的信息
print(os.listdir('.'))

['.ipynb_checkpoints', 'FoodPlace', 'Transportation']


In [25]:


# for each in os.listdir():  # 遍历每个文件
#     if each == '.ipynb_checkpoints':  # 忽略'.ipynb_checkpoints'文件夹
#         continue
#     try:
#         coord = get_coordinates(each)
#         # 将每个文件的信息作为字典添加到列表中
#         df_list.append({'FileName': each, 'Latitude': coord[0], 'Longitude': coord[1], 'Time': coord[2]})
#     except Exception as e:
#         print('文件{}无法解析: {}'.format(each, e))

for folder_path, subfolders, filenames in os.walk('FoodPlace/'):
    for each in filenames:
        if each.endswith('.ipynb_checkpoints'):  # 忽略 '.ipynb_checkpoints' 文件
            continue
        file_path = os.path.join(folder_path, each)  # 获取文件的完整路径
        try:
            coord = get_coordinates(file_path)
            if coord is not None:
                # 将每个文件的信息作为字典添加到列表中
                df_list.append({
                    'FolderName': os.path.basename(folder_path),  # 添加文件夹名
                    'FileName': each,
                    'Latitude': coord[0],
                    'Longitude': coord[1],
                    'Time': coord[2]
                })
            else:
                print(f"文件 {each} 缺少 GPS 信息")
        except Exception as e:
            print(f'文件{each}无法解析: {e}')

# 最后将列表转换为 DataFrame
df = pd.DataFrame(df_list)

EXIF 标签: {'Image Make': (0x010F) ASCII=Apple @ 146, 'Image Model': (0x0110) ASCII=iPhone 13 Pro @ 152, 'Image Orientation': (0x0112) Short=Horizontal (normal) @ 42, 'Image XResolution': (0x011A) Ratio=72 @ 166, 'Image YResolution': (0x011B) Ratio=72 @ 174, 'Image ResolutionUnit': (0x0128) Short=Pixels/Inch @ 78, 'Image Software': (0x0131) ASCII=18.0.1 @ 182, 'Image DateTime': (0x0132) ASCII=2024:11:05 13:19:10 @ 190, 'Image HostComputer': (0x013C) ASCII=iPhone 13 Pro @ 210, 'Image ExifOffset': (0x8769) Long=224 @ 126, 'GPS GPSLatitudeRef': (0x0001) ASCII=N @ 2644, 'GPS GPSLatitude': (0x0002) Ratio=[22, 18, 1911/100] @ 2820, 'GPS GPSLongitudeRef': (0x0003) ASCII=E @ 2668, 'GPS GPSLongitude': (0x0004) Ratio=[114, 10, 4633/100] @ 2844, 'GPS GPSAltitudeRef': (0x0005) Byte=0 @ 2692, 'GPS GPSAltitude': (0x0006) Ratio=994271/87718 @ 2868, 'GPS GPSTimeStamp': (0x0007) Ratio=[5, 19, 857/100] @ 2876, 'GPS GPSSpeedRef': (0x000C) ASCII=K @ 2728, 'GPS GPSSpeed': (0x000D) Ratio=0 @ 2900, 'GPS GPSImg

In [26]:
print(os.listdir('.'))

['.ipynb_checkpoints', 'FoodPlace', 'Transportation']


In [27]:
df

,FolderName,FileName,Latitude,Longitude,Time
0,Chill_Cup,Chill_Cup.jpg,22.305308,114.179536,2024:11:05 13:19:10
1,Chill_Cup,Chill_Cup2.jpg,22.305308,114.179536,2024:11:05 13:19:13
2,Chill_Cup,Chill_Cup3.jpg,22.305272,114.179550,2024:11:05 13:19:24
3,Communal_Student_Canteen,Communal_Student_Canteen.jpg,22.305044,114.180000,2024:11:05 14:07:38
4,Communal_Student_Canteen,Communal_Student_Canteen2.jpg,22.305208,114.179894,2024:11:05 14:07:54
5,Communal_Student_Canteen,Communal_Student_Canteen3.jpg,22.305189,114.179778,2024:11:05 14:08:10
6,Communal_Student_Canteen,Communal_Student_Canteen4.jpg,22.305200,114.179717,2024:11:05 14:08:20
7,Communal_Student_Restaurant,Communal_Student_Restaurant.3.jpg,22.304628,114.180039,2024:11:05 14:10:23
8,Communal_Student_Restaurant,Communal_Student_Restaurant.jpg,22.304883,114.180144,2024:11:05 14:09:36
9,Communal_Student_Restaurant,Communal_Student_Restaurant2.jpg,22.305247,114.180100,2024:11:05 14:09:28


In [28]:
# 返回上一级目录
os.chdir('../')
print(os.listdir('.'))

['.ipynb_checkpoints', '1.getLatitudeLongtitudeFromPhotos-Copy1.ipynb', '2.DrawMap-Copy1.ipynb', '2.DrawMap.ipynb', 'FileName&Infomation.csv', 'FoodPlace.csv', 'HTMLMAP.html', 'images', 'Step1getTransportaionLatitudeLongtitudeFromPhotos-Copy1.ipynb', 'Step2getFoodPlaceLatitudeLongtitudeFromPhotos.ipynb', 'Transportation.csv']


## 保存为表格 Save as table

In [29]:
df.to_csv('FoodPlace.csv', index=False, encoding='utf-8')